# 🔍 SERP API - Google Search Results

Test the SERP (Search Engine Results Page) API:
- Google Search
- Location-specific results
- Batch queries
- Device type comparison
- Pagination (fetching more than 10 results)

---

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

API_TOKEN = os.getenv("BRIGHTDATA_API_TOKEN")
if not API_TOKEN:
    raise ValueError("Set BRIGHTDATA_API_TOKEN in .env file")

print(f"API Token: {API_TOKEN[:10]}...{API_TOKEN[-4:]}")
print("Setup complete!")

API Token: 7011787d-2...3336
Setup complete!


## Initialize Client

In [4]:
from brightdata import BrightDataClient

# Initialize client
client = BrightDataClient(token=API_TOKEN)

print("Client initialized")
print(f"Default SERP zone: {client.serp_zone}")

           

Client initialized
Default SERP zone: sdk_serp


In [5]:
!pip show brightdata-sdk          

Name: brightdata-sdk
Version: 2.1.2
Summary: Modern async-first Python SDK for Bright Data APIs
Home-page: https://github.com/brightdata/sdk-python
Author: Bright Data
Author-email: Bright Data <support@brightdata.com>
License: MIT
Location: /Users/ns/Desktop/projects/sdk-python/.venv/lib/python3.11/site-packages
Editable project location: /Users/ns/Desktop/projects/sdk-python
Requires: aiohttp, aiolimiter, click, pydantic, pydantic-settings, python-dotenv, requests, tldextract
Required-by: 


---
## Test 1: Google Search

Basic Google search with default settings.

In [6]:
QUERY = "python programming tutorial"

print(f"Google Search: '{QUERY}'")
print("Location: United States\n")

async with client:
    result = await client.search.google(
        query=QUERY,
        location="United States",
        num_results=10
    )

print(f"Success: {result.success}")
print(f"Search Engine: {result.search_engine}")
print(f"Total Found: {result.total_found}")

if result.success and result.data:
    print("\n--- Top 5 Results ---")
    for i, item in enumerate(result.data[:5]):
        print(f"\n{i+1}. {item.get('title', 'N/A')}")
        print(f"   URL: {item.get('url', 'N/A')[:60]}...")
        desc = item.get('description', 'N/A')
        print(f"   Description: {desc[:80]}..." if len(str(desc)) > 80 else f"   Description: {desc}")
else:
    print(f"\nError: {result.error}")

Google Search: 'python programming tutorial'
Location: United States

Success: True
Search Engine: google
Total Found: None

--- Top 5 Results ---

1. The Python Tutorial — Python 3.14.3 documentation
   URL: https://docs.python.org/3/tutorial/index.html...
   Description: This tutorial introduces the reader informally to the basic concepts and feature...

2. Python Tutorial
   URL: https://www.w3schools.com/python/...
   Description: Learn Python. Python is a popular programming language. Python can be used on a ...

3. Python For Beginners
   URL: https://www.python.org/about/gettingstarted/...
   Description: Welcome! Are you completely new to programming? If not then we presume you will ...

4. Best Python tutorial for beginners in 2024? : r/learnpython
   URL: https://www.reddit.com/r/learnpython/comments/1ajlvog/best_p...
   Description: I'm almost done with The Complete Python Bootcamp from Zero to Hero in Python th...

5. Learn Python - Free Interactive Python Tutorial
   URL: 

---
## Test 2: Location-Specific Search

Get location-specific search results.

In [7]:
QUERY = "best restaurants near me"

locations = ["New York, United States", "London, United Kingdom", "Tokyo, Japan"]

print(f"Query: '{QUERY}'")
print("Testing different locations...\n")

async with client:
    for location in locations:
        print(f"=== Location: {location} ===")
        result = await client.search.google(
            query=QUERY,
            location=location,
            num_results=3
        )
        
        if result.success and result.data:
            for i, item in enumerate(result.data[:3]):
                title = item.get('title', 'N/A')[:60]
                url = item.get('url', 'N/A')
                print(f"  {i+1}. {title}")
                print(f"     {url}")
        else:
            print(f"  Error: {result.error}")
        print()

Query: 'best restaurants near me'
Testing different locations...

=== Location: New York, United States ===
  1. Best Restaurants Near Me
     https://www.tripadvisor.com/Restaurants
  2. Best Restaurants Near Me - February 2026
     https://www.yelp.com/nearme/restaurants
  3. The 10 Best Restaurants Near Me in New York City, NY
     https://www.opentable.com/nearby/restaurants-near-me-new-york-city

=== Location: London, United Kingdom ===
  1. Best Restaurants Near Me
     https://www.tripadvisor.com/Restaurants
  2. TOP 10 BEST Restaurants near Bentonville, AR
     https://www.yelp.com/c/bentonville-ar-us/restaurants
  3. The Best Restaurants Open Near Me - TheFork
     https://www.thefork.com/near-me

=== Location: Tokyo, Japan ===
  1. Best Restaurants Near Me
     https://www.tripadvisor.com/Restaurants
  2. 10 Best Punta Gorda Restaurants that Prove it's a Foodie ...
     https://rtwin30days.com/best-punta-gorda-restaurants/
  3. The Best Places to Eat in St. Augustine, Florida

---
## Test 3: Batch Queries

Search multiple queries at once.

In [8]:
QUERIES = [
    "python web scraping",
    "javascript async await",
    "data science tools"
]

print(f"Batch Google Search: {len(QUERIES)} queries\n")

async with client:
    results = await client.search.google(
        query=QUERIES,
        location="United States",
        num_results=5
    )

print(f"Results: {len(results)} responses\n")

for i, result in enumerate(results):
    query = QUERIES[i]
    print(f"=== Query: '{query}' ===")
    print(f"Success: {result.success}")
    
    if result.success and result.data:
        print(f"Results found: {len(result.data)}")
        for j, item in enumerate(result.data[:2]):
            title = item.get('title', 'N/A')[:60]
            url = item.get('url', 'N/A')
            print(f"  {j+1}. {title}")
            print(f"     {url}")
    else:
        print(f"Error: {result.error}")
    print()

Batch Google Search: 3 queries

Results: 3 responses

=== Query: 'python web scraping' ===
Success: True
Results found: 8
  1. Python Web Scraping Tutorial
     https://www.geeksforgeeks.org/python/python-web-scraping-tutorial/
  2. Beautiful Soup: Build a Web Scraper With Python
     https://realpython.com/beautiful-soup-web-scraper-python/

=== Query: 'javascript async await' ===
Success: True
Results found: 10
  1. async function - JavaScript - MDN Web Docs
     https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Statements/async_function
  2. JavaScript async and await
     https://www.w3schools.com/js/js_async.asp

=== Query: 'data science tools' ===
Success: True
Results found: 9
  1. 13 Essential Data Science Tools (And How to Use Them)
     https://learning.linkedin.com/resources/learning-tech/how-to-use-13-essential-data-science-tools
  2. If you had to list a “tier list” of software that data scien
     https://www.reddit.com/r/datascience/comments/184ezlq/if_yo

---
## Test 4: Device Type (Desktop vs Mobile)

Compare desktop and mobile search results.

In [9]:
QUERY = "weather today"

print(f"Query: '{QUERY}'")
print("Comparing desktop vs mobile results...\n")

async with client:
    # Desktop search
    print("=== Desktop ===")
    desktop_result = await client.search.google(
        query=QUERY,
        device="desktop",
        num_results=5
    )
    
    if desktop_result.success and desktop_result.data:
        for i, item in enumerate(desktop_result.data[:3]):
            print(f"  {i+1}. {item.get('title', 'N/A')[:50]}")
    else:
        print(f"  Error: {desktop_result.error}")
    
    print("\n=== Mobile ===")
    mobile_result = await client.search.google(
        query=QUERY,
        device="mobile",
        num_results=5
    )
    
    if mobile_result.success and mobile_result.data:
        for i, item in enumerate(mobile_result.data[:3]):
            print(f"  {i+1}. {item.get('title', 'N/A')[:50]}")

Query: 'weather today'
Comparing desktop vs mobile results...

=== Desktop ===
  1. National and Local Weather Radar, Daily Forecast, 
  2. Weather Forecast and Conditions for New York City,
  3. Weather Forecast and Conditions for Austin, Texas

=== Mobile ===
  1. Spain Current Weather
  2. Weather forecast by locations
  3. 10-Day Weather Forecast for Valdeacederas, Tetuán 


---
## Test 5: Timing Metadata

In [10]:
# Check timing metadata from last result
print("=== Result Metadata ===")
print(f"success: {mobile_result.success}")
print(f"search_engine: {mobile_result.search_engine}")
print(f"query: {mobile_result.query}")
print(f"country: {mobile_result.country}")
print(f"results_per_page: {mobile_result.results_per_page}")
print(f"total_found: {mobile_result.total_found}")
print("\n=== Timing ===")
print(f"trigger_sent_at: {mobile_result.trigger_sent_at}")
print(f"data_fetched_at: {mobile_result.data_fetched_at}")

if mobile_result.trigger_sent_at and mobile_result.data_fetched_at:
    duration = (mobile_result.data_fetched_at - mobile_result.trigger_sent_at).total_seconds()
    print(f"\nTotal time: {duration:.2f} seconds")

=== Result Metadata ===
success: True
search_engine: google
query: {'q': 'weather today', 'location': None, 'language': 'en'}
country: None
results_per_page: 5
total_found: None

=== Timing ===
trigger_sent_at: 2026-02-11 17:55:31.882102+00:00
data_fetched_at: 2026-02-11 17:55:36.380537+00:00

Total time: 4.50 seconds


---
## Test 6: Export Results to JSON

In [11]:
import json
from pathlib import Path

# Use the last successful result
if mobile_result.success and mobile_result.data:
    output_file = Path.cwd() / "serp_results.json"
    
    export_data = {
        "success": mobile_result.success,
        "search_engine": mobile_result.search_engine,
        "query": mobile_result.query,
        "total_found": mobile_result.total_found,
        "results_count": len(mobile_result.data),
        "results": mobile_result.data,
    }
    
    with open(output_file, "w") as f:
        json.dump(export_data, f, indent=2, default=str)
    
    print(f"Exported to: {output_file}")
    print(f"Results count: {len(mobile_result.data)}")
else:
    print("No successful results to export")

Exported to: /Users/ns/Desktop/projects/sdk-python/notebooks/serp_results.json
Results count: 10


---
## Test 7: Pagination (Fetching More Than 10 Results)

Test sequential pagination to fetch more than one page of results.
Google returns ~10 results per page, so requesting 30+ results triggers automatic pagination.

In [12]:
QUERY = "machine learning frameworks"
NUM_RESULTS = 30  # Request 30 results (3 pages)

print(f"Query: '{QUERY}'")
print(f"Requesting {NUM_RESULTS} results (pagination will fetch ~3 pages)...\n")

async with client:
    result = await client.search.google(
        query=QUERY,
        location="United States",
        num_results=NUM_RESULTS
    )

print(f"Success: {result.success}")
print(f"Results returned: {len(result.data) if result.data else 0}")
print(f"Total found (Google estimate): {result.total_found}")
print(f"Results per page: {result.results_per_page}")

if result.error:
    print(f"Note: {result.error}")

if result.success and result.data:
    print(f"\n--- All {len(result.data)} Results ---")
    for i, item in enumerate(result.data):
        title = item.get('title', 'N/A')[:55]
        print(f"{i+1:2}. {title}")
else:
    print(f"\nError: {result.error}")

Query: 'machine learning frameworks'
Requesting 30 results (pagination will fetch ~3 pages)...

Success: True
Results returned: 30
Total found (Google estimate): None
Results per page: 10

--- All 30 Results ---
 1. Top 10 Machine Learning Frameworks in 2025
 2. Machine Learning Frameworks and Languages
 3. Machine Learning Framework Preferences in the Industry
 4. Machine and Deep Learning Frameworks
 5. What Are Machine Learning Frameworks?
 6. Top 10 Machine Learning Frameworks (2026)
 7. Machine learning frameworks
 8. Machine Learning Frameworks: Features, Use Cases ...
 9. Top Machine Learning Frameworks To Use
10. What Is a Machine Learning Pipeline?
11. josephmisiti/awesome-machine-learning: A curated list o
12. Effective Machine Learning Frameworks
13. The Top 16 AI Frameworks and Libraries: A Beginner's Gu
14. What Are Machine Learning Frameworks and How to Pick ..
15. YouTube
16. Top 20 Machine Learning Frameworks Of All Times
17. Comparison of ML Frameworks
18. Machine lear

---
## Summary

### SERP Methods

| Method | Description |
|--------|-------------|
| `client.search.google(query, ...)` | Google Search |

### Parameters

| Parameter | Description | Default |
|-----------|-------------|--------|
| `query` | Search query or list of queries | Required |
| `location` | Geographic location | `None` |
| `language` | Language code (en, es, ru, etc.) | `"en"` |
| `device` | `"desktop"`, `"mobile"`, `"tablet"` | `"desktop"` |
| `num_results` | Number of results (pagination auto-enabled for >10) | `10` |
| `zone` | SERP zone | `sdk_serp` |

### Pagination

When `num_results > 10`, the SDK automatically paginates through multiple Google result pages:
- Google returns ~10 results per page
- SDK fetches pages sequentially until reaching `num_results`
- Maximum 20 pages (200 results) per request
- 5 minute timeout for paginated requests

### Response Fields

| Field | Description |
|-------|-------------|
| `success` | Boolean indicating success |
| `data` | List of search result items |
| `search_engine` | Which engine was used |
| `query` | The search query info |
| `total_found` | Google's estimated total results |
| `results_per_page` | Results per page (10) |
| `error` | Error message (set for partial failures) |

### Result Item Fields

| Field | Description |
|-------|-------------|
| `title` | Page title |
| `url` | Page URL |
| `description` | Snippet/description |
| `position` | Ranking position |